In [1]:
import numpy as np
import pandas as pd
import glob
import json
import hashlib
from datetime import timedelta

### Raw Data 불러오기

In [2]:
# filelist=glob.glob('../data/login_data_with_treat/*')[:2]

In [3]:
# df = pd.read_csv(filelist[0]).iloc[:,1:]
# df2=pd.read_csv(filelist[1]).iloc[:,1:]
# df=pd.concat([df, df2], axis=0)

In [57]:
df = pd.read_csv('total.csv')
df.inDate = pd.to_datetime(df.inDate.apply(lambda x: x[:-1])) + timedelta(hours=9) # KST
# df['gamer_id'] = df['gamer_id'].apply(lambda x : hashlib.sha256(x.split('-')[1].encode()).hexdigest())
df['Login'] = 1
df = df[['game_id', 'gamer_id', 'inDate', 'Login']]


In [58]:
df.head()

,game_id,gamer_id,inDate,Login
0,1442,gamer_9234,2021-07-01 09:00:54.174,1
1,1442,gamer_9234,2021-07-01 09:01:02.105,1
2,2147,gamer_8506,2021-07-01 09:01:15.138,1
3,1046,gamer_1599,2021-07-01 09:03:27.895,1
4,1482,gamer_7833,2021-07-01 09:06:04.362,1


### 데이터 만드는 함수

In [59]:
#게임-유저 별로 사용될 시간 프레임 만들기
#start=시작날짜, end='끝나는날짜', freq='몇시간/몇일 등 간격 정하기', periods='몇개를 만들건지'(end와 함께 쓸 수 없음)
def make_dateframe(start=None ,end=None, freq=None, periods=None):
    
    dates=pd.date_range(start=start ,end=end, periods=periods, freq=freq)
    str_dates=[str(d) for d in dates]
    
    #행:날짜(시간) 열:범주화한 url 2개, IAP, Login
    date_frame=pd.DataFrame(np.zeros((len(str_dates),4), dtype=int), \
                columns=['social_url', 'non_social_url', 'IAP', 'Login'], index=str_dates)
    return date_frame

### 데이터프레임, 데이터 형태 만들기

In [60]:
lag='1H' #일단 지금은 데이터가 별로 없어서 1시간 단위로 설정

#나중에 더 추가할 예정
if 'H' in lag:
    if len(lag)==2:
        hour_hop=int(lag[0])
        df['time']=df['inDate'].apply(lambda x:x.strftime('%Y-%m-%d %H:00:00'))

In [64]:
date_frame=make_dateframe(start='2021-07-01', end='2021-10-13', freq=lag)

In [65]:
date_frame

,social_url,non_social_url,IAP,Login
2021-07-01 00:00:00,0,0,0,0
2021-07-01 01:00:00,0,0,0,0
2021-07-01 02:00:00,0,0,0,0
2021-07-01 03:00:00,0,0,0,0
2021-07-01 04:00:00,0,0,0,0
...,...,...,...,...
2021-10-12 20:00:00,0,0,0,0
2021-10-12 21:00:00,0,0,0,0
2021-10-12 22:00:00,0,0,0,0
2021-10-12 23:00:00,0,0,0,0


In [66]:
#데이터프레임 예시
df

,game_id,gamer_id,inDate,Login,time
0,1442,gamer_9234,2021-07-01 09:00:54.174,1,2021-07-01 09:00:00
1,1442,gamer_9234,2021-07-01 09:01:02.105,1,2021-07-01 09:00:00
2,2147,gamer_8506,2021-07-01 09:01:15.138,1,2021-07-01 09:00:00
3,1046,gamer_1599,2021-07-01 09:03:27.895,1,2021-07-01 09:00:00
4,1482,gamer_7833,2021-07-01 09:06:04.362,1,2021-07-01 09:00:00
...,...,...,...,...,...
189980,1572,gamer_9968,2021-10-13 10:43:58.708,1,2021-10-13 10:00:00
189981,1908,gamer_3677,2021-10-13 10:44:07.646,1,2021-10-13 10:00:00
189982,1908,gamer_3677,2021-10-13 10:44:18.560,1,2021-10-13 10:00:00
189983,1572,gamer_6099,2021-10-13 10:45:53.676,1,2021-10-13 10:00:00


### 형식에 맞춰서 데이터 넣기

In [67]:
#전체 게임, 유저 리스트
game_gamer_ls=list(set([(game, gamer_id) for (game, gamer_id) in zip(df.game_id, df.gamer_id)]))

#게임-유저의 시간대별 합
groupby=df.groupby(['game_id','gamer_id', 'time']).sum()[['Login']].reset_index()

In [68]:
groupby.groupby('game_id').sum()

,Login
game_id,
72,1419
85,217
120,53
215,21
232,22
...,...
2147484420,2
2147484527,345
2147484610,5


In [69]:
#게임{유저:데이터프레임} 형식 만들기
for i in range(len(game_gamer_ls)):
    item=game_gamer_ls[i]
    if i==0:
        to_dict={item[0]:{item[1]:date_frame.copy()}}

    else:
        
        #game_id만 있을 때
        if item[0] in to_dict.keys():
            to_dict[item[0]].update({item[1]:date_frame.copy()})
            
        #game_id가 없을 때:
        elif item[0] not in to_dict.keys():
            to_dict[item[0]]={item[1]:date_frame.copy()}
        

In [70]:
from tqdm import tqdm
for i in tqdm(range(len(groupby))):
    login=groupby.iloc[i]
    temp=to_dict[login['game_id']][login['gamer_id']].copy()
    
    if login['time'] in temp.index:
        temp.loc[login['time'], 'Login']=login['Login']
        to_dict[login[0]][login[1]]=temp
    
    else:
        pass   

100%|██████████| 84035/84035 [00:13<00:00, 6271.22it/s]


In [84]:
to_dict[772]["gamer_4126"].Login[to_dict[772]["gamer_4126"].Login!=0]

2021-08-21 03:00:00    2
Name: Login, dtype: int64

In [85]:
game_and_push = [
                ('2021-08-24', 2195),
                ('2021-09-21', 1773),
                ('2021-09-24', 1773),
                ('2021-09-27', 1802),
                ('2021-09-29', 2516)
                ]
dic = {}
for push_date, game in game_and_push:
    users = to_dict[game].keys()
    user_list = []
    for user in tqdm(users, total=len(users)):
        cur_user = {}
        data = to_dict[game][user][['Login']]
        temp = pd.to_datetime(data.index)

        ## t == 1 group
        start_t1 = temp.searchsorted(pd.to_datetime(push_date) - timedelta(days=14))
        push = temp.searchsorted(pd.to_datetime(push_date))
        end_t1 = temp.searchsorted(pd.to_datetime(push_date) + timedelta(days=7))

        ## t == 0 group
        start_t0 = temp.searchsorted(pd.to_datetime(push_date) - timedelta(days=30) - timedelta(days=14))
        not_push = temp.searchsorted(pd.to_datetime(push_date) - timedelta(days=30))
        end_t0 = temp.searchsorted(pd.to_datetime(push_date) - timedelta(days=30) + timedelta(days=7))
        

        # data.index = data.index.strftime('%Y-%m-%d %H:00:00')

        X_t1 = data.iloc[start_t1:push]
        y_t1 = data.iloc[push:end_t1]

        X_t1['treatment'] = 1
        
        
        response = y_t1[y_t1['Login'] > 0]
        y_t1 = 1 if len(response) > 0 else 0

        X_t0 = data.iloc[start_t0:not_push]
        y_t0 = data.iloc[not_push:end_t0]

        X_t0['treatment'] = 1
        
        
        response = y_t0[y_t0['Login'] > 0]
        y_t0 = 1 if len(response) > 0 else 0

        # X_t1.index.name = 'time'
        # X_t0.index.name = 'time'

        # X_t1.reset_index(inplace=True)
        # X_t0.reset_index(inplace=True)
        
        cur_user['X_t1'] = X_t1.to_json(orient='table', indent=4)
        cur_user['y_t1'] = y_t1
        cur_user['X_t0'] = X_t0.to_json(orient='table', indent=4)
        cur_user['y_t0'] = y_t0



        user_list.append(cur_user)
    dic[game] = user_list

  0%|          | 0/414 [00:00<?, ?it/s]/tmp/ipykernel_234363/2635700435.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_t1['treatment'] = 1
/tmp/ipykernel_234363/2635700435.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_t0['treatment'] = 1
100%|██████████| 21/21 [00:00<00:00, 214.32it/s]


In [86]:
import json

with open("data_revised.json", "w") as json_file:
    json.dump(dic, json_file, indent=4)

In [ ]:
#json 타입으로 저장하기
#dataframe -> dictionary
out_dict={}
for idx in game_gamer_ls:
    temp=to_dict[idx[0]][idx[1]]
    dic=temp.to_dict()
    out_dict[idx[0]]={idx[1]:dic}

import json
with open('input_data_login.json','w')as f:
    json.dump(out_dict, f, indent=4)

In [ ]:
#pickle 타입으로 저장하기 (용량 큼)
import pickle
with open('input_data_login_pickle','wb')as f:
    pickle.dump(out_dict, f)